# Removing Clouds and Clouds Shadows from Landsat8 imagery
**Problem:** Some regions are covered mostly through clouds such that selecting the least cloudy image isn't enough.

To overcome this problem, one can mask out cloudy pixels in a collection and create a composite:
1. Mask cloudy pixels.
2. Check if a single image is cloud free.
3. If not, create a composite.

In [2]:
import ee
import geemap

EE_PROJECT_NAME = 'seeing-the-big-picture'

try:
    ee.Authenticate()
    ee.Initialize(project=EE_PROJECT_NAME)
except Exception as e:
    print("Please authenticate Earth Engine: earthengine authenticate")
    raise

# Rectange surrounding the training sample 'airport 32' from fMoW
bbox = [-79.48694022837181, -1.0140575646754744,
        -79.4420222533541, -0.9691395896577617]

center_coords = [-0.99, -79.46]

region = ee.Geometry.Rectangle(bbox)

def scale_l8(image):
    return (image
            .select(['SR_B2', 'SR_B3', 'SR_B4'])
            .multiply(0.0000275)
            .add(-0.2))

l8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
       .filterBounds(region)
       .map(scale_l8))

ModuleNotFoundError: No module named 'ee'

In [ ]:
def mask_l8_clouds(image):
    qa = image.select('QA_PIXEL')   # See https://www.usgs.gov/landsat-missions/landsat-collection-2-quality-assessment-bands
    cloud_mask = qa.bitwiseAnd(int('11111', 2)).eq(0)  # Mask away non clear pixels.
    return image.updateMask(cloud_mask)

l8_cloudyless = l8.map(mask_l8_clouds)

def count_valid(image):
    return image.select('SR_B2').mask().reduce(ee.Reducer.sum()) 

scol_with_count = l8_cloudyless.map(lambda img: img.set('valid_pixels', count_valid(img)))
least_cloudy = scol_with_count.sort('valid_pixels', False).first()  # Most valid pixels

m = geemap.Map(center=center_coords, zoom=13)
m.addLayer(
    least_cloudy,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    'Least Cloudy'
)
m.addLayerControl()
m